<a href="https://colab.research.google.com/github/coder272377/dvp/blob/develop/tfew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top Five Equal Weight



In [ ]:
interval = "5m"
previous_days = 5
tickers = "AAPL TSLA GOOG AMZN MSFT".split()



## Package management



In [ ]:
import datetime
import functools
import math

from typing import List, Tuple, Union



In [ ]:
Version = Union[List[int], Tuple]



In [ ]:
def check_version(req:Version, version_str:str):
  v0, v1, v2 = version_str.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



How do I unload (reload) a Python module? https://stackoverflow.com/questions/437589/how-do-i-unload-reload-a-python-module



In [ ]:
import pandas as pd
if not check_version([1, 3, 5], pd.__version__):
  !pip install --upgrade pandas
  import importlib
  importlib.reload(pd)
  assert check_version([1, 3, 5], pd.__version__)



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

Download Financial Dataset Using Yahoo Finance in Python | A Complete Guide https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/



yfinance Library – A Complete Guide https://algotrading101.com/learn/yfinance-guide/


In [ ]:
import pandas as pd



In [ ]:
try:
  import yfinance as yf
except ModuleNotFoundError:
  !pip install yfinance
  !pip install yahoofinancials
  import yfinance as yf



Charting Candlestick_OHLC one minute bars with Pandas and Matplotlib https://stackoverflow.com/questions/41821916/charting-candlestick-ohlc-one-minute-bars-with-pandas-and-matplotlib

Candlestick Chart in Python (mplfinance, plotly, bokeh, bqplot and cufflinks) https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh#1

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import scipy.signal as signal



## Data period

In [ ]:
@functools.lru_cache()
def tz_ny() -> str:
  return "America/New_York"



Most recent previous business day in Python https://stackoverflow.com/questions/2224742/most-recent-previous-business-day-in-python and https://stackoverflow.com/a/51528258



In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
@functools.lru_cache()
def USBday():
  return pd.tseries.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())



pandas.Series.dt.tz_convert https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.tz_convert.html

In [ ]:
@functools.lru_cache()
def today(tz=tz_ny()) -> pd.Timestamp.date:
  now = pd.Timestamp.today(tz=tz)

  if now.hour < 10:
    now += pd.Timedelta(-1, unit='D')

  return now.date()



In [ ]:
@functools.lru_cache()
def get_start_date(n_days:int, end_date=today(), unit=USBday()):
  return end_date + unit * (-n_days)



Python for Finance, Part I: Yahoo & Google Finance API, pandas, and matplotlib
https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

An Even Easier Guide to Getting Stock Data With Python
https://towardsdatascience.com/an-even-easier-guide-to-getting-stock-data-with-python-1a109df6b593



Google Finance API and Its Alternatives (Updated for 2021) https://rapidapi.com/blog/google-finance-api-alternatives/

## Data-getters



In [ ]:
@functools.lru_cache()
def get_df_start_end(ticker:str, start:pd.Timestamp, end:pd.Timestamp=today(), interval:str=interval):
  assert start < end
  if interval[-1] == 'm':
    assert (pd.Timedelta(60) > pd.Timestamp.today(tz=tz_ny()) - start), f"{start} older than 60 days for interval {interval!r}"
  tic = yf.Ticker(ticker)
  return tic.history(start=start, end=end, interval=interval)



In [ ]:
def get_df_one_day(ticker:str, date:pd.Timestamp.date, interval:str=interval) -> pd.DataFrame:
  start = pd.Timestamp(date, unit="s")
  end = pd.Timestamp(date+pd.Timedelta(1, 'D'), unit="s") + pd.Timedelta(-1, 's')
  return get_df_start_end(ticker=ticker, start=start, end=end, interval=interval)



In [ ]:
def get_df(ticker:str, n_days:int, end_date:pd.Timestamp=today(), interval:str=interval):
  return get_df_start_end(ticker, get_start_date(n_days, end_date=end_date), end=end_date, interval=interval)



In [ ]:
def get_days_df(ticker:str, n_days:int=previous_days, end_date:pd.Timestamp=today(), interval:str="1D") -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



In [ ]:
def get_minutes_df(ticker:str, n_days:int=2, end_date:pd.Timestamp=today(), interval:str=interval) -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



Test data getter functions



In [ ]:
def test_get_days_df_sunday():
  tic = "GOOG"
  days = 5
  end_date = pd.Timestamp("2022-01-23 10:50", tz=tz_ny())
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert pd.Timestamp("2022-01-21 00:00:00") == df_days.index[-1]



In [ ]:
test_get_days_df_sunday()



In [ ]:
def test_get_days_df__holiday():
  tic = "MSFT"
  days = 5
  end_date = pd.Timestamp("2021-06-02 09:50", tz=tz_ny())
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert pd.Timestamp("2021-06-01 00:00:00") == df_days.index[-1], df_days



In [ ]:
test_get_days_df__holiday()

